In [213]:
import pandas as pd

In [214]:
commission_rate = 0.00
match_name = "raj_neil"

In [215]:
df = pd.read_csv(f'{match_name}.csv')

# Display the first few rowss to verify
print("Data Preview:")
df

Data Preview:


,Player,Side,Wager
0,Harsh,Neil,20
1,Darshak,Raj,20
2,Aditya,Neil,20
3,Subhajeet,Raj,20
4,Arin,Raj,20


In [216]:
sides = df['Side'].unique()
sides

array(['Neil', 'Raj'], dtype=object)

In [217]:
totals = df.groupby('Side')['Wager'].sum()
total_A = totals.get(sides[0], 0)
total_B = totals.get(sides[1], 0)

print(f"\nTotal wager on {sides[0]}: {total_A}")
print(f"Total wager on {sides[1]}: {total_B}")


Total wager on Neil: 40
Total wager on Raj: 60


In [218]:
def calculate_payout(row, sides, total_A, total_B, commission_rate=0.05):
    """
    Calculate the payout for a player, taking into account the commission fee.

    :param row: The row containing the player's details (including 'Side' and 'Wager').
    :param sides: List of two sides ('Side A' and 'Side B').
    :param total_A: Total amount wagered on Side A.
    :param total_B: Total amount wagered on Side B.
    :param commission_rate: The commission fee percentage (default is 5%).
    :return: The net payout after commission.
    """
    side = row['Side']
    wager = row['Wager']
    
    # Calculate the payout based on the side that wins
    if side == sides[0]:  # Player bet on Side A
        payout = round(wager / total_A * total_B, 2) if total_A > 0 else 0
    elif side == sides[1]:  # Player bet on Side B
        payout = round(wager / total_B * total_A, 2) if total_B > 0 else 0
    else:
        return 0
    
    # Apply the commission fee
    commission = payout * commission_rate
    net_payout = round(payout - commission, 2)
    
    return net_payout


In [219]:
def calculate_gross_payout(row, sides, total_A, total_B, commission_rate=0.05):
    side = row['Side']
    wager = row['Wager']
    
    # Calculate the payout based on the side that wins
    if side == sides[0]:  # Player bet on Side A
        payout = round(wager / total_A * total_B, 2) if total_A > 0 else 0
    elif side == sides[1]:  # Player bet on Side B
        payout = round(wager / total_B * total_A, 2) if total_B > 0 else 0
    else:
        return 0
    
    # Apply the commission fee
    commission = payout * commission_rate
    net_payout = round(payout - commission, 2)
    
    return net_payout + row['Wager']

In [220]:
df[f'Payout_if_{sides[0]}_wins'] = df.apply(lambda row: calculate_payout(row, sides, total_A, total_B, commission_rate) if row['Side'] == sides[0] else 0, axis=1)
df[f'Payout_if_{sides[1]}_wins'] = df.apply(lambda row: calculate_payout(row, sides, total_A, total_B, commission_rate) if row['Side'] == sides[1] else 0, axis=1)

In [221]:
df[f'Total_Payout_if_{sides[0]}_wins'] = df.apply(lambda row: calculate_gross_payout(row, sides, total_A, total_B, commission_rate) if row['Side'] == sides[0] else 0, axis=1)
df[f'Total_Payout_if_{sides[1]}_wins'] = df.apply(lambda row: calculate_gross_payout(row, sides, total_A, total_B, commission_rate) if row['Side'] == sides[1] else 0, axis=1)

In [222]:
df.to_csv(f"{match_name}_payout.csv", index=False)
df

,Player,Side,Wager,Payout_if_Neil_wins,Payout_if_Raj_wins,Total_Payout_if_Neil_wins,Total_Payout_if_Raj_wins
0,Harsh,Neil,20,30.0,0.00,50.0,0.00
1,Darshak,Raj,20,0.0,13.33,0.0,33.33
2,Aditya,Neil,20,30.0,0.00,50.0,0.00
3,Subhajeet,Raj,20,0.0,13.33,0.0,33.33
4,Arin,Raj,20,0.0,13.33,0.0,33.33
